<a href="https://colab.research.google.com/github/BooEiEi/Master-Project/blob/main/Hyperparameter_Lightgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import os# os ใช้จัดการกับไฟล์ ว่าอยู่ที่ไหน
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path_DF = '/content/drive/My Drive/master project/data/DF'
path_DF2 = '/content/drive/My Drive/master project/data/DF2'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import lightgbm
import sklearn
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error

In [52]:
df_META = pd.read_csv(os.path.join(path_DF, 'df_META.csv'))
df_AAPL= pd.read_csv(os.path.join(path_DF, 'df_AAPL.csv'))
df_MSFT= pd.read_csv(os.path.join(path_DF, 'df_MSFT.csv'))
df_NVDA= pd.read_csv(os.path.join(path_DF, 'df_NVDA.csv'))
df_GOOG= pd.read_csv(os.path.join(path_DF2, 'df_GOOG.csv'))

In [21]:
df_META=df_META.drop(['Unnamed: 0'], axis=1)
df_META['date'] = pd.to_datetime(df_META['date'])
for lag in [1, 3, 5, 7]:
    df_META[f'lag_{lag}'] = df_META['Stockprice_META'].shift(lag)

# ลบค่าที่เป็น NaN จากการสร้าง lag features
df_META.dropna(inplace=True)

In [27]:
df_AAPL=df_AAPL.drop(['Unnamed: 0'], axis=1)
df_AAPL['date'] = pd.to_datetime(df_AAPL['date'])
for lag in [1, 3, 5, 7]:
    df_AAPL[f'lag_{lag}'] = df_AAPL['Stockprice_AAPL'].shift(lag)

# ลบค่าที่เป็น NaN จากการสร้าง lag features
df_AAPL.dropna(inplace=True)

In [32]:
df_MSFT=df_MSFT.drop(['Unnamed: 0'], axis=1)
df_MSFT['date'] = pd.to_datetime(df_MSFT['date'])
for lag in [1, 3, 5, 7]:
    df_MSFT[f'lag_{lag}'] = df_MSFT['Stockprice_MSFT'].shift(lag)

# ลบค่าที่เป็น NaN จากการสร้าง lag features
df_MSFT.dropna(inplace=True)

In [33]:
df_NVDA=df_NVDA.drop(['Unnamed: 0'], axis=1)
df_NVDA['date'] = pd.to_datetime(df_NVDA['date'])
for lag in [1, 3, 5, 7]:
    df_NVDA[f'lag_{lag}'] = df_NVDA['Stockprice_NVDA'].shift(lag)

# ลบค่าที่เป็น NaN จากการสร้าง lag features
df_NVDA.dropna(inplace=True)

In [53]:
df_GOOG=df_GOOG.drop(['Unnamed: 0'], axis=1)
df_GOOG['date'] = pd.to_datetime(df_GOOG['date'])

In [47]:
train_size = int(len(df_META) * 0.7)  # 70% ของ 753
test_size = len(df_META) - train_size  # 30% ที่เหลือ

# META

## Train Test

In [23]:
# แบ่งข้อมูลโดยไม่สุ่ม (คงลำดับเวลา)
train_META = df_META.iloc[:train_size]  # ข้อมูล train
test_META = df_META.iloc[train_size:]  # ข้อมูล test

### Train Varidation

In [24]:
X_META = train_META.drop(columns=["Stockprice_META","date","RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_META = train_META["Stockprice_META"]

In [25]:
X_META

,sentiment_META,Interest,UNEMPLOYMENT,CPI,US_Treasury_Yield,US_inflation_rate,Real_GDP,S&P500,NASDAQ,BTC,Gold,lag_1,lag_3,lag_5,lag_7
7,0.406150,4.33,4.0,317.671,4.53,0.030,6001.465,6144.15,20056.25,96644.8,2936.1,694.84,668.13,673.70,668.20
8,0.536325,4.33,4.0,317.671,4.55,0.030,6001.465,6129.58,20041.26,95638.0,2949.0,703.77,683.55,657.50,658.24
9,0.399740,4.33,4.0,317.671,4.47,0.030,6001.465,6114.63,20026.77,97504.6,2900.7,716.37,694.84,668.13,673.70
10,-0.726900,4.33,4.0,317.671,4.52,0.030,6001.465,6115.07,19945.64,96610.2,2945.4,736.67,703.77,683.55,657.50
11,0.304667,4.33,4.0,317.671,4.62,0.030,6001.465,6051.97,19649.95,97881.3,2928.7,728.56,716.37,694.84,668.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,0.179971,4.33,3.5,299.170,3.49,0.065,5469.175,4060.43,11512.41,23016.0,1930.0,151.74,148.97,188.77,186.06
525,0.009677,4.33,3.5,299.170,3.46,0.065,5469.175,4016.22,11313.36,23055.1,1942.6,147.30,147.06,153.12,186.53
526,0.321800,4.33,3.5,299.170,3.46,0.065,5469.175,4016.95,11334.27,22632.5,1935.4,141.50,151.74,148.97,188.77
527,-0.177067,4.33,3.5,299.170,3.52,0.065,5469.175,4019.81,11364.41,22915.5,1928.6,143.14,147.30,147.06,153.12


In [26]:
model = LGBMRegressor(random_state=42)

# Define the hyperparameter search space
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_data_in_leaf': [25, 50, 100, 150, 200],
    'n_estimators' : [100, 200, 400, 600],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'max_bin' : [50, 100, 150, 200, 400]
}

# Create a TimeSeriesSplit object for time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search object on the training data
grid_search.fit(X_META , y_META)

# Retrieve the best model and best hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Evaluate the best model's performance on the test set
y_pred = best_model.predict(X_META)
mse = mean_squared_error(y_META, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")

[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000209 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1750
[LightGBM] [Info] Number of data points in the train set: 522, number of used features: 15
[LightGBM] [Info] Start training from score 408.657797
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warn

#AAPL

In [28]:
# แบ่งข้อมูลโดยไม่สุ่ม (คงลำดับเวลา)
train_AAPL = df_AAPL.iloc[:train_size]  # ข้อมูล train
test_AAPL = df_AAPL.iloc[train_size:]  # ข้อมูล test

In [29]:
X_AAPL = train_AAPL.drop(columns=["Stockprice_AAPL","date", "Interest", "US_Treasury_Yield", "RETAIL_SALES", "US_Dollar_Index"])
y_AAPL = train_AAPL["Stockprice_AAPL"]

In [31]:
model = LGBMRegressor(random_state=42)

# Define the hyperparameter search space
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_data_in_leaf': [25, 50, 100, 150, 200],
    'n_estimators' : [100, 200, 400, 600],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'max_bin' : [50, 100, 150, 200, 400]
}

# Create a TimeSeriesSplit object for time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search object on the training data
grid_search.fit(X_AAPL , y_AAPL)

# Retrieve the best model and best hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Evaluate the best model's performance on the test set
y_pred = best_model.predict(X_AAPL)
mse = mean_squared_error(y_AAPL, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")

[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000114 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 564
[LightGBM] [Info] Number of data points in the train set: 522, number of used features: 14
[LightGBM] [Info] Start training from score 194.054751
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warni

#MSFT

In [35]:
# แบ่งข้อมูลโดยไม่สุ่ม (คงลำดับเวลา)
train_MSFT = df_MSFT.iloc[:train_size]  # ข้อมูล train
test_MSFT = df_MSFT.iloc[train_size:]  # ข้อมูล test

In [36]:
X_MSFT = train_MSFT.drop(columns=["Stockprice_MSFT","date","RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_MSFT = train_MSFT["Stockprice_MSFT"]

In [37]:
model = LGBMRegressor(random_state=42)

# Define the hyperparameter search space
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_data_in_leaf': [25, 50, 100, 150, 200],
    'n_estimators' : [100, 200, 400, 600],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'max_bin' : [50, 100, 150, 200, 400]
}

# Create a TimeSeriesSplit object for time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search object on the training data
grid_search.fit(X_MSFT , y_MSFT)

# Retrieve the best model and best hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Evaluate the best model's performance on the test set
y_pred = best_model.predict(X_MSFT)
mse = mean_squared_error(y_MSFT, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")

[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000172 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1072
[LightGBM] [Info] Number of data points in the train set: 522, number of used features: 15
[LightGBM] [Info] Start training from score 373.791638
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warn

#NVDA

In [38]:
# แบ่งข้อมูลโดยไม่สุ่ม (คงลำดับเวลา)
train_NVDA = df_NVDA.iloc[:train_size]  # ข้อมูล train
test_NVDA = df_NVDA.iloc[train_size:]  # ข้อมูล test

In [39]:
X_NVDA = train_NVDA[["sentiment_NVDA", "Interest", "CPI", "BTC", "S&P500", "lag_1", "lag_3", "lag_5", "lag_7"]]
y_NVDA = train_NVDA["Stockprice_NVDA"]

In [40]:
model = LGBMRegressor(random_state=42)

# Define the hyperparameter search space
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_data_in_leaf': [25, 50, 100, 150, 200],
    'n_estimators' : [100, 200, 400, 600],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'max_bin' : [50, 100, 150, 200, 400]
}

# Create a TimeSeriesSplit object for time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search object on the training data
grid_search.fit(X_NVDA , y_NVDA)

# Retrieve the best model and best hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Evaluate the best model's performance on the test set
y_pred = best_model.predict(X_NVDA)
mse = mean_squared_error(y_NVDA, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")

[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000103 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 384
[LightGBM] [Info] Number of data points in the train set: 522, number of used features: 9
[LightGBM] [Info] Start training from score 387.247701
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warnin

#GOOG

In [54]:
# แบ่งข้อมูลโดยไม่สุ่ม (คงลำดับเวลา)
train_GOOG = df_GOOG.iloc[:train_size]  # ข้อมูล train
test_GOOG = df_GOOG.iloc[train_size:]  # ข้อมูล test

In [56]:
X_GOOG = train_GOOG.drop(columns=["Stockprice_GOOGL","date", "Interest", "RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_GOOG = train_GOOG["Stockprice_GOOGL"]

In [57]:
model = LGBMRegressor(random_state=42)

# Define the hyperparameter search space
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_data_in_leaf': [25, 50, 100, 150, 200],
    'n_estimators' : [100, 200, 400, 600],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'max_bin' : [50, 100, 150, 200, 400]
}

# Create a TimeSeriesSplit object for time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search object on the training data
grid_search.fit(X_GOOG , y_GOOG)

# Retrieve the best model and best hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Evaluate the best model's performance on the test set
y_pred = best_model.predict(X_GOOG)
mse = mean_squared_error(y_GOOG, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")

[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000217 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1064
[LightGBM] [Info] Number of data points in the train set: 527, number of used features: 14
[LightGBM] [Info] Start training from score 145.233444
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warn